## Scheduling task:
We have n people's availabilities for 9a, 10a, 11a, 12, 1p, 2p, 3p, 4p defined as a vector of 0 and 1's
We would like to schedule $J$ meetings between different groups of people, represented by $J$ index sets $\mathcal{I_j}\subseteq\{1,\dots,n\}$.

Rules:
* Each meeting $\mathcal{I}_j$ must occur at one time $t$
* All people attending meeting $\mathcal{I}_j$ must be available at time $t$
* All people attending meeting $\mathcal{I}_j$ must not be attending another meeting at time $t$
* No attendee should have >2 hours of consecutive meetings.

In [32]:
using BooleanSatisfiability

n = 5 # number of people
T = 8 # number of times
# nxT matrix: each row is one attendee's meeting availability
A_bar = [
    1 0 1 0 1 1 1 1
    0 1 1 0 0 0 0 1
    1 1 1 0 1 1 0 1
    1 1 0 1 1 0 0 0
    0 1 1 1 0 0 0 1
]
index_sets = [[1,2,3], [3,4,5], [1,3,5], [1,4]]
J = length(index_sets) # number of meetings

4

Define $\bar a_{it}\in\{0,1\}$ = the known availability of person $i$ at time $t$ and $\bar A\in\{0,1\}^{n\times T}$.

Define $a_{it} \in\{0,1\}$ = 1 if person $i$ is attending a meeting at time $t$, 0 otherwise. $A\in\{0,1\}^{n\times T}$.

Define $m_j\in\{0,1\}^T$ = the time at which meeting $\mathcal{I}_j$ is scheduled, in one-hot encoding.

Since $\bar A$ is known, the goal is to find a satisfying assignment of $A$ and $m_1,\dots,m_J$ such that all meetings can occur following the given rules.

* Participants cannot attend meetings when they are unavailable.
$$\bigwedge_{it} \bar a_{it} \implies \neg a_{it}$$

In [33]:
@satvariable(A[1:n, 1:T], Bool)

#past_availability = all(¬A_bar .⟹ ¬A)
booked = BoolExpr[]
for i=1:n
    for t=1:T
        if A_bar[i,t]==0
            push!(booked, ¬A[i,t])
        end
    end
end
unavailability = all(booked)

AND_81fff29a008ded48
 | NOT_71aac0f717453642
 |  | A_1_2
 | NOT_aa6196e5d4cc26af
 |  | A_1_4
 | NOT_cf2c14a0b2535f8
 |  | A_2_1
 | NOT_826349772eba559b
 |  | A_2_4
 | NOT_1bac913b12a338bc
 |  | A_2_5
 | NOT_547e26f28fb4c097
 |  | A_2_6
 | NOT_2262b4a859b33800
 |  | A_2_7
 | NOT_384097f73d28f91
 |  | A_3_4
 | NOT_60b022ebd3fc95e0
 |  | A_3_7
 | NOT_65ebb7bfd84897b7
 |  | A_4_3
 | NOT_bf350ccb950b79ac
 |  | A_4_6
 | NOT_569a776aa6467d1c
 |  | A_4_7
 | NOT_d9bf741be6fa2852
 |  | A_4_8
 | NOT_5c95bf02c25d8a4a
 |  | A_5_1
 | NOT_b0741e715f7a7528
 |  | A_5_5
 | NOT_9aacfd0327980298
 |  | A_5_6
 | NOT_d5f0d95ec3b95bec
 |  | A_5_7


* Define $m_{j,t}$ = 1 if meeting $j$ can occur at time $t$ and 0 otherwise.
$$m_{j,t} := \bigwedge_{i\in\mathcal{I}_j} a_{it}$$

In [34]:
# M_{j,t} = 1 if meeting j can be scheduled at time t
M = [and(A[index_sets[j], t]) for j=1:J, t=1:T]

4×8 Matrix{BoolExpr}:
 AND_518a1de68820ae6f
 | A_1_1
 | A_2_1
 | A_3_1
  …  AND_60f36fb73cb68ec7
 | A_1_8
 | A_2_8
 | A_3_8

 AND_5ec388e3aac31c31
 | A_3_1
 | A_4_1
 | A_5_1
     AND_abdff09862b286b6
 | A_3_8
 | A_4_8
 | A_5_8

 AND_5507d71cae6de07d
 | A_1_1
 | A_3_1
 | A_5_1
     AND_e7d735eb88228ce8
 | A_1_8
 | A_3_8
 | A_5_8

 AND_638eb935e9366a47
 | A_1_1
 | A_4_1
              AND_842955db82b9e118
 | A_1_8
 | A_4_8


* For each meeting, everyone must be available at some time $t$ and not attending another meeting.
$$\bigwedge_{j,t} m_{j,t}\implies \neg\left(\bigvee_{j'\neq j} m_{j',t}\right)$$

In [35]:
# for each meeting j, everyone must be available at some time t and not attending another meeting
# get a list of conflicts
conflicts = [filter((i) -> i != j && length(intersect(index_sets[j], index_sets[i])) > 0, 1:J) for j=1:J ]

no_double_booking = all([M[j,t] ⟹ ¬or(M[conflicts[j],t]) for j=1:J, t=1:T])

AND_8517aab0f5dced5
 | OR_aa83f78075bf8452
 |  | NOT_b7a1b6969f5ddff3
 |  |  | AND_518a1de68820ae6f
 |  |  |  | A_1_1
 |  |  |  | A_2_1
 |  |  |  | A_3_1
 |  | NOT_1a8008c2ee05f93f
 |  |  | OR_a60c5cb57ae6e87f
 |  |  |  | AND_5ec388e3aac31c31
 |  |  |  |  | A_3_1
 |  |  |  |  | A_4_1
 |  |  |  |  | A_5_1
 |  |  |  | AND_5507d71cae6de07d
 |  |  |  |  | A_1_1
 |  |  |  |  | A_3_1
 |  |  |  |  | A_5_1
 |  |  |  | AND_638eb935e9366a47
 |  |  |  |  | A_1_1
 |  |  |  |  | A_4_1
 | OR_51175ce6f44ea4b3
 |  | NOT_5cd4f1529c2f5e23
 |  |  | AND_5ec388e3aac31c31
 |  |  |  | A_3_1
 |  |  |  | A_4_1
 |  |  |  | A_5_1
 |  | NOT_909ce9650a433c8
 |  |  | OR_bac84395ca1f5581
 |  |  |  | AND_518a1de68820ae6f
 |  |  |  |  | A_1_1
 |  |  |  |  | A_2_1
 |  |  |  |  | A_3_1
 |  |  |  | AND_5507d71cae6de07d
 |  |  |  |  | A_1_1
 |  |  |  |  | A_3_1
 |  |  |  |  | A_5_1
 |  |  |  | AND_638eb935e9366a47
 |  |  |  |  | A_1_1
 |  |  |  |  | A_4_1
 | OR_d6f20116f7318fec
 |  | NOT_10141e96f2cc0643
 |  |  | AND_5507

* All meetings must have at least one valid time.
$$\bigwedge_j\bigvee_t m_{j,t}$$

In [36]:
# all meetings must be scheduled
require_one_time = all([or(M[j,:]) for j=1:J])

AND_434aeacc99a87ad7
 | OR_3a4c1b52bf30a5a9
 |  | AND_518a1de68820ae6f
 |  |  | A_1_1
 |  |  | A_2_1
 |  |  | A_3_1
 |  | AND_7f0c5048b0ecf086
 |  |  | A_1_2
 |  |  | A_2_2
 |  |  | A_3_2
 |  | AND_7c43cc52cedf3766
 |  |  | A_1_3
 |  |  | A_2_3
 |  |  | A_3_3
 |  | AND_7a5e401263fa14c2
 |  |  | A_1_4
 |  |  | A_2_4
 |  |  | A_3_4
 |  | AND_96137b8e83312cbe
 |  |  | A_1_5
 |  |  | A_2_5
 |  |  | A_3_5
 |  | AND_860966636f421fa6
 |  |  | A_1_6
 |  |  | A_2_6
 |  |  | A_3_6
 |  | AND_d955a1fc831575d2
 |  |  | A_1_7
 |  |  | A_2_7
 |  |  | A_3_7
 |  | AND_60f36fb73cb68ec7
 |  |  | A_1_8
 |  |  | A_2_8
 |  |  | A_3_8
 | OR_54494c4b4ea088d2
 |  | AND_5ec388e3aac31c31
 |  |  | A_3_1
 |  |  | A_4_1
 |  |  | A_5_1
 |  | AND_4a68ed75d1c45180
 |  |  | A_3_2
 |  |  | A_4_2
 |  |  | A_5_2
 |  | AND_e029da222d0952c2
 |  |  | A_3_3
 |  |  | A_4_3
 |  |  | A_5_3
 |  | AND_8606a8bcf1d3857d
 |  |  | A_3_4
 |  |  | A_4_4
 |  |  | A_5_4
 |  | AND_158831e251498c3b
 |  |  | A_3_5
 |  |  | A_4_5
 |  |  | A_5

* No attendee should have more than two consecutive hours of meetings.
$$\bigwedge_i \bigwedge_{t=1,\dots,T-2} \neg(a_{i,t}\wedge a_{i,t+1} \wedge a_{i,t+2})$$

In [37]:
# nobody should have more than 2 consecutive hours of meetings
time_limit = all([¬and(A[i,t:t+2]) for i=1:n, t=1:T-2])

AND_d90e58815f040dd5
 | NOT_c9179f19f5024f76
 |  | AND_9842cc730ff366c4
 |  |  | A_1_1
 |  |  | A_1_2
 |  |  | A_1_3
 | NOT_3894fdbf71d3e760
 |  | AND_8251988d7749e1ca
 |  |  | A_2_1
 |  |  | A_2_2
 |  |  | A_2_3
 | NOT_78e7d4380262ec16
 |  | AND_e72c6edfeaf78500
 |  |  | A_3_1
 |  |  | A_3_2
 |  |  | A_3_3
 | NOT_491cfd72123536fc
 |  | AND_c73f24b8dd6af83e
 |  |  | A_4_1
 |  |  | A_4_2
 |  |  | A_4_3
 | NOT_1e12d603f6c77deb
 |  | AND_33dbca13eb9caf1f
 |  |  | A_5_1
 |  |  | A_5_2
 |  |  | A_5_3
 | NOT_cdf139b91870eeb6
 |  | AND_29788cb84c19aca0
 |  |  | A_1_2
 |  |  | A_1_3
 |  |  | A_1_4
 | NOT_5a78e8fd3c944312
 |  | AND_db63f5cf4c4c2233
 |  |  | A_2_2
 |  |  | A_2_3
 |  |  | A_2_4
 | NOT_c9e35a96ff9cb46e
 |  | AND_c640dcfdd2ada9ea
 |  |  | A_3_2
 |  |  | A_3_3
 |  |  | A_3_4
 | NOT_7d33765f49b9c9c5
 |  | AND_2f57346120dbb07
 |  |  | A_4_2
 |  |  | A_4_3
 |  |  | A_4_4
 | NOT_fcf31c28ee656c2
 |  | AND_e96c4a42e5e6abcf
 |  |  | A_5_2
 |  |  | A_5_3
 |  |  | A_5_4
 | NOT_b9b6d0327e2ace

### Solve the problem

In [39]:
# solve
exprs = [no_double_booking, require_one_time, unavailability, time_limit]

status = sat!(exprs)
# 1. generate the SMT file
# 2. call z3 on the file and parse the output
# 3. set the value of the variable A and the expressions constructed from A (M, time_limit, etc)

:SAT

In [40]:
value(A)

5×8 Matrix{Bool}:
 1  0  1  0  1  0  0  1
 0  0  0  0  0  0  0  1
 0  1  1  0  0  0  0  1
 1  1  0  0  1  0  0  0
 0  1  1  0  0  0  0  0

In [42]:
value(M)

4×8 Matrix{Bool}:
 0  0  0  0  0  0  0  1
 0  1  0  0  0  0  0  0
 0  0  1  0  0  0  0  0
 1  0  0  0  1  0  0  0

In [43]:
times = ["9a", "10a", "11a", "12p", "1p", "2p", "3p", "4p"]
for j=1:J
    println("Meeting with attendees $(index_sets[j]) can occur at $(times[findall(value(M[j,:]) .== true)])")
end

Meeting with attendees [1, 2, 3] can occur at ["4p"]
Meeting with attendees [3, 4, 5] can occur at ["10a"]
Meeting with attendees [1, 3, 5] can occur at ["11a"]
Meeting with attendees [1, 4] can occur at ["9a", "1p"]


## Now let's see what this would look like in SMTLIB2

In [44]:
for (i,line) in enumerate(readlines(open("out.smt", "r")))
    println(i, " ", line)
end

1 (declare-const A_1_1 Bool)
2 (declare-const A_2_1 Bool)
3 (declare-const A_3_1 Bool)
4 (declare-const A_4_1 Bool)
5 (declare-const A_5_1 Bool)
6 (declare-const A_1_2 Bool)
7 (declare-const A_2_2 Bool)
8 (declare-const A_3_2 Bool)
9 (declare-const A_4_2 Bool)
10 (declare-const A_5_2 Bool)
11 (declare-const A_1_3 Bool)
12 (declare-const A_2_3 Bool)
13 (declare-const A_3_3 Bool)
14 (declare-const A_4_3 Bool)
15 (declare-const A_5_3 Bool)
16 (declare-const A_1_4 Bool)
17 (declare-const A_2_4 Bool)
18 (declare-const A_3_4 Bool)
19 (declare-const A_4_4 Bool)
20 (declare-const A_5_4 Bool)
21 (declare-const A_1_5 Bool)
22 (declare-const A_2_5 Bool)
23 (declare-const A_3_5 Bool)
24 (declare-const A_4_5 Bool)
25 (declare-const A_5_5 Bool)
26 (declare-const A_1_6 Bool)
27 (declare-const A_2_6 Bool)
28 (declare-const A_3_6 Bool)
29 (declare-const A_4_6 Bool)
30 (declare-const A_5_6 Bool)
31 (declare-const A_1_7 Bool)
32 (declare-const A_2_7 Bool)
33 (declare-const A_3_7 Bool)
34 (declare-const A

151 (define-fun NOT_9264710450eb8c9f () Bool (not OR_7871457c3154e710))
152 (define-fun OR_d8a9d702a609a382 () Bool (or NOT_79f2886729e88ab3 NOT_9264710450eb8c9f))
153 (define-fun NOT_1c2cefc126c902d7 () Bool (not AND_4062d73dca4d5d12))
154 (define-fun OR_cdf67547eb70ee84 () Bool (or AND_860966636f421fa6 AND_dfd2ecf0aae8e276 AND_e2f8ae31a22f5652))
155 (define-fun NOT_2b327b8e420e9640 () Bool (not OR_cdf67547eb70ee84))
156 (define-fun OR_d9f7d0f9f34194b9 () Bool (or NOT_1c2cefc126c902d7 NOT_2b327b8e420e9640))
157 (define-fun NOT_c9af2981eccf9059 () Bool (not AND_e2f8ae31a22f5652))
158 (define-fun OR_2e84572846790ecf () Bool (or AND_4062d73dca4d5d12 AND_860966636f421fa6 AND_dfd2ecf0aae8e276))
159 (define-fun NOT_aad9c6ae7202ebbf () Bool (not OR_2e84572846790ecf))
160 (define-fun OR_b6ad964a804ab1c4 () Bool (or NOT_aad9c6ae7202ebbf NOT_c9af2981eccf9059))
161 (define-fun AND_d955a1fc831575d2 () Bool (and A_1_7 A_2_7 A_3_7))
162 (define-fun NOT_50e63c498b136ebe () Bool (not AND_d955a1fc8315

267 (define-fun AND_9010c2478e70cc6f () Bool (and A_2_5 A_2_6 A_2_7))
268 (define-fun NOT_a6ce8f7bbb5e2c6f () Bool (not AND_9010c2478e70cc6f))
269 (define-fun AND_13a445e018117fa8 () Bool (and A_3_5 A_3_6 A_3_7))
270 (define-fun NOT_c684198b7119a0cb () Bool (not AND_13a445e018117fa8))
271 (define-fun AND_73b09a2530d85435 () Bool (and A_4_5 A_4_6 A_4_7))
272 (define-fun NOT_267fca51d9a4c8d6 () Bool (not AND_73b09a2530d85435))
273 (define-fun AND_4234253dabfea4cb () Bool (and A_5_5 A_5_6 A_5_7))
274 (define-fun NOT_c5cbf85bff79fbe3 () Bool (not AND_4234253dabfea4cb))
275 (define-fun AND_352a781e1a478827 () Bool (and A_1_6 A_1_7 A_1_8))
276 (define-fun NOT_ce4c105161ebdc1b () Bool (not AND_352a781e1a478827))
277 (define-fun AND_a5cbaf2f2a57029c () Bool (and A_2_6 A_2_7 A_2_8))
278 (define-fun NOT_e82a922f51f64b4d () Bool (not AND_a5cbaf2f2a57029c))
279 (define-fun AND_8303c5c59ff68ec2 () Bool (and A_3_6 A_3_7 A_3_8))
280 (define-fun NOT_81fb87b5c7fe9288 () Bool (not AND_8303c5c59ff68ec2))